# Define variables

We can move the variables that won't change to the `workflow_executor`

In [ ]:
from src import workflow_executor

In [ ]:
# Cloud functions must be deployed to retrieve it name
# The name of the cloud function must be "af-update-experiment-status"
function_metadata = ! gcloud functions describe af-update-experiment-status --gen2 --region us-central1 --format="value(serviceConfig.uri)"
function_uri = function_metadata[0]

### Project, GCS and job ID

In [ ]:
project_id = 'rl-alphafold-dev'     # Your project ID
bucket_name = 'rl-alphafold-dbs'    # Bucket name
region = 'us-central1'              # Region

local_sequence_path = 'sequences/seq1.fasta'    # Path to your local sequence
image_uri = f'gcr.io/{project_id}/alphafold'    # Path to image on GCR

In [ ]:
# Change to the desired machine type
data_pipeline_machine_type = 'c2-standard-16'
predict_relax_machine_type = 'n1-standard-8-t4'

### Network

In [ ]:
network_name = 'alphafold'          # Change to your network name
subnetwork_name = 'alphafold-nw'    # Change to your subnetwork name

nfs_ip_address = '10.130.0.2'       # Filestore IP
nfs_path = '/datasets'              # Path to NFS share

### Data Pipeline

In [ ]:
max_template_date = '2020-05-14'
model_preset = 'monomer'        
db_preset = 'reduced_dbs'       

### Predict and Relax

In [ ]:
# Change to the location (parent path) where the parameters are stored (GCS)
model_params_path = 'rl-alphafold-dbs'

In [ ]:
# Change only if necessary
num_predictions_per_model = 1
run_relax = True
random_seed = 1000
parallelism = 5
num_models = 5

In [ ]:
# Define you own labels
labels = {
    'run_name': 'run_1',
    'protein_name': 't1050'
}

# Execute Workflow

In [ ]:
args = workflow_executor.prepare_args_for_experiments(
    project_id = project_id,
    bucket_name = bucket_name,
    region = region,
    function_uri = function_uri,
    local_sequence_path = local_sequence_path,
    data_pipeline_machine_type = data_pipeline_machine_type,
    predict_relax_machine_type = predict_relax_machine_type,
    image_uri = image_uri,
    network_name = network_name,
    subnetwork_name = subnetwork_name,
    nfs_ip_address = nfs_ip_address,
    nfs_path = nfs_path,
    max_template_date = max_template_date,
    model_preset = model_preset,
    db_preset = db_preset,
    num_predictions_per_model = num_predictions_per_model,
    model_params_path = model_params_path,
    num_models = num_models,
    run_relax = run_relax,
    random_seed = random_seed,
    parallelism = parallelism,
    labels=labels)

In [ ]:
workflow_executor.execute_workflow('alphafold-workflow-batch', args)